In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset= pd.read_csv(r"D:\HopeAI\MLC-Week4\10_RandomForest_Grid_Classification\Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [24]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier       

In [14]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
             'max_features':['auto','sqrt','log2'],
             'n_estimators':[10,100]}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3, n_jobs=-1, scoring = 'f1')

grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\mohan\anaconda3\Lib\site-packages\sklearn\base.py", line 471, in _validate_params
    

,estimator,RandomForestClassifier()
,param_grid,"{'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'sqrt', ...], 'n_estimators': [10, 100]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,None
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,10


In [15]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
        

In [16]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, grid_predictions, average = 'weighted')
print("The f1_macro value for the best parameter {}:".format(grid.best_params_),f1_macro)
        

The f1_macro value for the best parameter {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 10}: 0.9108102345415777


In [17]:
print("The Confusion Matrix:\n",cm)


The Confusion Matrix:
 [[78  7]
 [ 5 44]]


In [18]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93        85
           1       0.86      0.90      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.90       134
weighted avg       0.91      0.91      0.91       134



In [19]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(x_test)[:,1])


0.9521008403361344

In [20]:
table = pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007548,0.003036,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.006390,0.002315,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.284854,0.037324,0.074383,0.023057,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.800000,0.871795,0.800000,0.923077,0.888889,0.856752,0.049193,1
3,2.747723,0.074272,0.155226,0.021048,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.727273,0.842105,0.809524,0.926829,0.947368,0.850620,0.080180,3
4,0.331023,0.022641,0.082963,0.037816,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.764706,0.842105,0.780488,0.809524,0.888889,0.817142,0.044546,7
5,2.769181,0.073808,0.168868,0.013037,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.800000,0.810811,0.809524,0.900000,0.947368,0.853541,0.059316,2
6,0.008450,0.007160,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.009897,0.007372,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.292822,0.047151,0.087163,0.016291,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.727273,0.842105,0.780488,0.871795,0.857143,0.815761,0.054075,8
9,2.496182,0.046532,0.092149,0.040881,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.764706,0.842105,0.809524,0.900000,0.923077,0.847882,0.057954,5


In [23]:
age_input = float(input("Age:"))
salary_input = float(input("Salary Expected:"))
sex_male_input = int(input("Sex Male 0 or 1:"))
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction=", format(Future_Prediction))

Age: 20
Salary Expected: 543.6
Sex Male 0 or 1: 1


Future_Prediction= [1]
